In [ ]:
import librosa
import numpy as np
import hashlib
from scipy.signal import find_peaks
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv("datos_financieros.csv")

# Extraer la columna "Close" (precio de cierre)
close_price = df["Close"].to_numpy()

# Configuración del algoritmo
NFFT = 2048   # Tamaño de la ventana de Fourier
HOP = 1024    # Salto de la ventana de Fourier
FS = 1        # Frecuencia de muestreo (no utilizada en este caso)
HASHBITS = 20 # Número de bits para la huella digital

# Calcular el espectrograma y guardar en disco duro
spectrogram = np.abs(librosa.stft(close_price, n_fft=NFFT, hop_length=HOP))**2
np.save('spectrogram.npy', spectrogram)

# Cargar el espectrograma desde disco duro
spectrogram = np.load('spectrogram.npy')

# Aplicar una transformación logarítmica al espectrograma para convertirlo en una escala de decibelios
log_spectrogram = librosa.power_to_db(spectrogram)

# Calcular los picos en el espectrograma utilizando la función find_peaks() de scipy
peaks, _ = find_peaks(log_spectrogram.reshape(-1), height=0)

# Redimensionar los picos a su forma original
time_indices = peaks % log_spectrogram.shape[1]
freq_indices = peaks // log_spectrogram.shape[1]
peaks = librosa.fft_frequencies(sr=FS, n_fft=NFFT)[freq_indices]

# Generar las huellas digitales a partir de los picos y calcular un hash único para cada huella digital
hashes = []
for i in range(len(peaks)):
    for j in range(i+1, len(peaks)):
        freq_i = peaks[i]
        freq_j = peaks[j]
        time_delta = time_indices[j] - time_indices[i]
        # Crear la huella digital a partir de los precios de cierre, volumen y datos de alto y bajo de los intervalos de tiempo
        close_i = close_price[time_indices[i]:time_indices[i]+NFFT]
        close_j = close_price[time_indices[j]:time_indices[j]+NFFT]
        volume_i = df["Volumen"][time_indices[i]]
        volume_j = df["Volumen"][time_indices[j]]
        high_i = df["High"][time_indices[i]]
        high_j = df["High"][time_indices[j]]
        low_i = df["Low"][time_indices[i]]
        low_j = df["Low"][time_indices[j]]
        hash = hashlib.sha1(f"{freq_i}:{freq_j}:{time_delta}:{close_i}:{close_j}:{volume_i}:{volume_j}:{high_i}:{high_j}:{low_i}:{low_j}".encode("utf-8")).hexdigest()[:HASHBITS//4]
        hashes.append(hash)

# Imprimir las huellas digitales generadas
print(hashes)